In [1]:
import pandas as pd
import requests
import json
import pymysql
import getpass

# Translate `product_category_name`s from Portuguese to English

1. Get a list of all `product_category_name`s currently in our database
2. Send API requet to IBM Watson to translate each word in this list
3. Process the response

## 1. get unique product categories

In [2]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=getpass.getpass(),
                       db="olist")

········


In [37]:
sql = """
SELECT DISTINCT 
    product_category_name
FROM products
WHERE product_category_name IS NOT NULL;
"""
product_categories = pd.read_sql(sql, conn)

In [38]:
product_categories = product_categories.assign(cats=product_categories["product_category_name"].str.replace("_", " "))

In [39]:
product_categories

,product_category_name,cats
0,perfumaria,perfumaria
1,artes,artes
2,esporte_lazer,esporte lazer
3,bebes,bebes
4,utilidades_domesticas,utilidades domesticas
...,...,...
68,casa_conforto_2,casa conforto 2
69,portateis_cozinha_e_preparadores_de_alimentos,portateis cozinha e preparadores de alimentos
70,seguros_e_servicos,seguros e servicos
71,moveis_colchao_e_estofado,moveis colchao e estofado


## 2. test API requests

In [16]:
auth = ("apikey", "aRHG9pdRpofv7m50m6ikFY-yDgsUG5mJfhnGVV4IGgUY")
headers = {"Content-Type": "application/json"}
url = "https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/d39ad4d8-67ff-4e11-9d6b-72ac4ae95d14/v3/translate?version=2018-05-01"

In [28]:
data = {"text": ["Todo Bem?", "algo mais"], "model_id": "pt-en"}

In [21]:
json.dumps(data)

'{"text": ["Todo Bem"], "model_id": "pt-en"}'

In [22]:
requests.post(url=url,
              headers=headers, 
              auth=auth, 
              data=json.dumps(data))

<Response [200]>

In [29]:
response = requests.post(url=url,
                         headers=headers,
                         auth=auth,
                         data=json.dumps(data))

In [30]:
response.json()

{'translations': [{'translation': 'All Right?'},
  {'translation': 'something more'}],
 'word_count': 4,
 'character_count': 18}

In [31]:
def translate(to_translate):
    auth = ("apikey", "aRHG9pdRpofv7m50m6ikFY-yDgsUG5mJfhnGVV4IGgUY")
    headers = {"Content-Type": "application/json"}
    url = "https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/d39ad4d8-67ff-4e11-9d6b-72ac4ae95d14/v3/translate?version=2018-05-01"
    
    data = {"text": to_translate, "model_id": "pt-en"}
    
    response = requests.post(url=url,
                             headers=headers,
                             auth=auth,
                             data=json.dumps(data))
    
    return response.json()["translations"]

In [33]:
a = ["Todo Bem?", "algo mais"]
pd.DataFrame(translate(a))

,translation
0,All Right?
1,something more


## 3. Combine the two components

In [41]:
to_translate = product_categories["cats"].to_list()

In [44]:
translations = translate(to_translate)

In [48]:
product_categories = product_categories.assign(english_cats=pd.DataFrame(translations)["translation"])

In [49]:
product_categories

,product_category_name,cats,english_cats
0,perfumaria,perfumaria,perfumery
1,artes,artes,arts
2,esporte_lazer,esporte lazer,sport leisure
3,bebes,bebes,babies
4,utilidades_domesticas,utilidades domesticas,domestical utilities
...,...,...,...
68,casa_conforto_2,casa conforto 2,house comfort 2
69,portateis_cozinha_e_preparadores_de_alimentos,portateis cozinha e preparadores de alimentos,portable cooking and food preparers
70,seguros_e_servicos,seguros e servicos,safe and servic
71,moveis_colchao_e_estofado,moveis colchao e estofado,movable collchao and upholstery
